# Classificação dados infomoney-data-clean

Nesse notebook vamos pegar os dados do arquivo 'infomoney-data-clean.csv' obtidos por meio de web scraping e classificaremos eles por meio do modelo disponível em https://huggingface.co/lucas-leme/FinBERT-PT-BR.

Depois, precisamos importar os pacotes que utilizaremos.

In [ ]:
import requests
import pandas as pd
import math as m
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

Após isso, precisamos ler os dados do nosso arquivo 'infomoney-data-clean.csv'. Para isso executamos o comando:

In [ ]:
df = pd.read_csv('/work/DadosWebScrapping/infomoney-data-clean.csv')
df

Para a classificação dos dados utilizaremos somente os títulos e subtítulos. Assim para selecionar somente a coluna dos títulos, executamos o comando:

In [ ]:
titles = df.loc[:, df.columns == 'title']
titles

E para selecionar somente a coluna dos subtítulos, executamos o comando:

In [ ]:
subtitles = df.loc[:, df.columns == 'subtitle']
subtitles

Depois disso, precisamos configurar nosso modelo que classificará os textos. Para isso definimos qual é o caminho até os arquivos de nosso modelo:

In [ ]:
path = '/work/ModeloHug'

Após isso, passamos o path para as classes AutoModelForSequenceClassification e AutoTokenizer para carregar o modelo pré-treinado e seu tokenizer associado. O tokenizer é o responsável pelas entradas do modelo. 

Além disso,  criamos um pipeline passando seu tipo ("sentiment-analysis") o modelo e o tokenizer. Um pipeline é um objeto que abstrai boa parte dos códigos complexos da biblioteca e facilita a utilização do modelo.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(path)

model = AutoModelForSequenceClassification.from_pretrained(path)

classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

Para utilizar o modelo criado, basta digitar:

In [ ]:
classifier("Frase que deseja analisar") 

Exemplo real de uma notícia do mercado financeiro:

In [ ]:
print(classifier("Ibovespa atinge recorde histórico e anima investidores"))

Como podemos ver, o resultado mais provável é de que a notícia é positiva. Agora vamos fazer isso para todos os títulos e subtítulos de notícias do nosso banco de dados. 

Começaremos pelos títulos. Para fazer isso criamos uma lista (results) para guardar os resultados do modelo. Depois criamos um laço que percorrerá todos os títulos, fará a classificação dele e adicionará o resultado dessa classificação na lista results. 

In [ ]:
# Obs: A execução dessa célula demorou 1h e 20 minutos aproximadamente
resultsTitles = []
for x in range(11136):
    # print(x)
    if type(titles['title'][x]) is str:
        result = (classifier(titles['title'][x]))[0]['label']
        resultsTitles.append(result)
    else:
        resultsTitles.append('NaN')

Fazemos a mesma coisa com os subtítulos:

In [ ]:
resultsSubTitles = []
for x in range(11136):
    # print(x)
    if type(subtitles['subtitle'][x]) is str:
        result = (classifier(subtitles['subtitle'][x]))[0]['label']
        resultsSubTitles.append(result)
    else:
        resultsSubTitles.append('NaN')

Após isso, adicionamos mais duas colunas ao nosso banco de dados inicial com os resultados obtidos.

In [ ]:
df['classTitles'] = resultsTitles
df['classSubTitles'] = resultsSubTitles
df

Para obter maior confiabilidade nos nossos dados, faremos uma comparação da classificação obtida pelo título e subtítulo de uma mesma notícia.

Para fazer isso, começaremos a selecionar as duas colunas referentes as classificações.

In [ ]:
classification = df.loc[:,['classTitles', 'classSubTitles']]
classification

Depois criamos um laço que passa por todas as 'linhas' da nossa tabela anterior, compara as duas classificações e guarda essa informação numa lista (resultsMatch). Além disso ela guarda a quantidade de cada resultado (de Miss, Match, NaN).

In [ ]:
qtdNaN = 0
qtdMatch = 0
qtdMiss = 0
resultsMatch = []
for x in range(11136):
    if((type(classification['classTitles'][x]) is str) and (type(classification['classSubTitles'][x]) is str)):
        if(classification['classTitles'][x] == "NaN" or classification['classSubTitles'][x] == "NaN"):
            qtdNaN += 1
            resultsMatch.append("NaN")
        elif(classification['classTitles'][x] == classification['classSubTitles'][x]):
            qtdMatch += 1
            resultsMatch.append("Match")
        else:
            qtdMiss += 1
            resultsMatch.append("Miss")
    else:
        qtdNaN += 1
        resultsMatch.append("NaN")
total = qtdMatch + qtdMiss + qtdNaN
tabela = [["","Match", "Miss", "NaN", "Total"], ["Quantidade", qtdMatch , qtdMiss, qtdNaN, total], ["Proporção", "%.2f%c" % ((qtdMatch/total * 100), "%"), "%.2f%c" % ((qtdMiss/total * 100), "%"), "%.2f%c" % ((qtdNaN/total * 100), "%"), "%.0f%c" % ((total/total * 100), "%")]]
matchBanco = pd.DataFrame(tabela)
matchBanco

Depois disso, precisamos guardar as informações de match em nosso banco. Para fazer isso adicionamos mais uma coluna em nossa tabela

In [ ]:
df['Match'] = resultsMatch
df

Por fim, criamos um novo arquivo para salvar o novo banco.

In [ ]:
df.to_csv('/work/DadosClassificados/infomoney-data-cleanCLASSIFIED.csv')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4246ec96-5bda-435e-947c-91f08e9f7576' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>